# Get metadata for records containing spreadsheet files in Zenodo
This code calls the Zenodo API and outputs the results to a JSON file.

Resources: instructions on the [Zenodo developers site](https://developers.zenodo.org/?python#quickstart-upload) and this [instruction for looping through API page results](https://viniciusdesouza-becker.medium.com/making-a-request-in-a-paginated-api-using-python-7be6279c2aca)

In [1]:
import requests
import re
from datetime import datetime
import json

In [ ]:
#enter the Zenodo API access token
#ACCESS_TOKEN = input("enter Zenodo access token: ")

In [2]:
#set up some lists
total_results = []
#set up current date and time for output filename
filetime = datetime.now()
filetime = filetime.strftime('%Y-%m-%d_%I-%M_%p')
#path - change if you want to save output to a different folder
path = '/Users/ap180/spreadsheets-experiments/'

In [3]:
#date_ranges = ['[2013 TO 2024}']
#test date ranges
#date_ranges = ['[2013 TO 2020}', '[2020 TO 2021}']
#round 1 date ranges:
#date_ranges = ['[2013 TO 2020}', '[2020 TO 2021}', '[2021 TO 2022}', '[2022-01-01 TO 2022-07-01}', '[2022-07-01 TO 2023}', '[2023-01-01 TO 2023-07-01}', '[2023-07-01 TO 2024}', '[2024-01-01 TO 2024-06-01}', '[2024-06-01 TO 2024-08-01}', '[2024-08-01 TO 2025}']
#round 2 date ranges (2019-2024)
date_ranges = ['[2019 TO 2020}', '[2020 TO 2021}', '[2021 TO 2022}', '[2022-01-01 TO 2022-07-01}', '[2022-07-01 TO 2023}', '[2023-01-01 TO 2023-07-01}', '[2023-07-01 TO 2024}', '[2024-01-01 TO 2024-06-01}', '[2024-06-01 TO 2024-08-01}', '[2024-08-01 TO 2025}']

In [ ]:
restrictedurl = 'https://zenodo.org/api/records'
restrictedparams = {'q': 'filetype:(csv OR xlsx OR tsv OR xls OR ods OR xlsb OR ots OR xlsm OR xltm OR xltx OR ogw) AND doi:zenodo AND created:[2019 TO 2025}',
    "size": "100",
    #"access_token": ACCESS_TOKEN
    #test q 'q': 'filetype: xlsb AND doi:zenodo'
    #first round q 'q': f'filetype:(csv OR xlsx OR tsv OR xls OR ods OR xlsb) AND doi:zenodo AND publicationdate:{date}'
    #expanded file formats q 
   }
response_restricted = requests.get(restrictedurl, params = restrictedparams)
print(response_restricted.url)

In [4]:
for date in date_ranges:
    url = 'https://zenodo.org/api/records'
    params = {'q': f'filetype:(csv OR xlsx OR tsv OR xls OR ods OR xlsb OR ots OR xlsm OR xltm OR xltx OR ogw) AND doi:zenodo AND created:{date}',
        "size": "100",
        #"access_token": ACCESS_TOKEN
        #test q 'q': 'filetype: xlsb AND doi:zenodo'
        #first round q 'q': f'filetype:(csv OR xlsx OR tsv OR xls OR ods OR xlsb) AND doi:zenodo AND publicationdate:{date}'
        #second round q 'q': f'filetype:(ots OR xlsm OR xltm OR xltx OR ogw) AND doi:zenodo AND publicationdate:{date}'
        #third round q f'filetype:(csv OR xlsx OR tsv OR xls OR ods OR xlsb OR ots OR xlsm OR xltm OR xltx OR ogw) AND doi:zenodo AND publicationdate:{date}'
       }
    response = requests.get(url, params = params)
    print(response.url)
    results = response.json()
    print(f'current page: {response.url}')
    print(f'the links on this current page are: {results["links"]}')
    total_results.extend(results['hits']['hits'])
    print(f'{len(results["hits"]["hits"])} records were added to the big list')
    nexturl = results['links']['next']
    #nextparams = {'access_token': ACCESS_TOKEN}
    print(f'setting next page: {nexturl}')
    #a while loop that gets all the pages of API results using the 'next' value in the result
    while True:
        response = requests.get(nexturl)
        print(f'current page {response.url}')
        if response.status_code != 200:
            raise ValueError('Request failed with status code ' + str(response.status_code) + 'problem url: ' + str(response.url))
            print(response.url)
            print(date)
        results = response.json()
        print(f'the links on this current page are: {results["links"]}')
        total_results.extend(results['hits']['hits'])
        print(f'{len(results["hits"]["hits"])} records were added to the big list')
        if 'next' in results['links'].keys():
            nexturl = results['links']['next']
            print(f'setting next page: {nexturl}')
        else:
            break
    print("We have", len(total_results), "total results")

https://zenodo.org/api/records?q=filetype%3A%28csv+OR+xlsx+OR+tsv+OR+xls+OR+ods+OR+xlsb+OR+ots+OR+xlsm+OR+xltm+OR+xltx+OR+ogw%29+AND+doi%3Azenodo+AND+created%3A%5B2019+TO+2020%7D&size=100
current page: https://zenodo.org/api/records?q=filetype%3A%28csv+OR+xlsx+OR+tsv+OR+xls+OR+ods+OR+xlsb+OR+ots+OR+xlsm+OR+xltm+OR+xltx+OR+ogw%29+AND+doi%3Azenodo+AND+created%3A%5B2019+TO+2020%7D&size=100
the links on this current page are: {'self': 'https://zenodo.org/api/records?page=1&q=filetype%3A%28csv%20OR%20xlsx%20OR%20tsv%20OR%20xls%20OR%20ods%20OR%20xlsb%20OR%20ots%20OR%20xlsm%20OR%20xltm%20OR%20xltx%20OR%20ogw%29%20AND%20doi%3Azenodo%20AND%20created%3A%5B2019%20TO%202020%7D&size=100&sort=bestmatch', 'next': 'https://zenodo.org/api/records?page=2&q=filetype%3A%28csv%20OR%20xlsx%20OR%20tsv%20OR%20xls%20OR%20ods%20OR%20xlsb%20OR%20ots%20OR%20xlsm%20OR%20xltm%20OR%20xltx%20OR%20ogw%29%20AND%20doi%3Azenodo%20AND%20created%3A%5B2019%20TO%202020%7D&size=100&sort=bestmatch'}
100 records were added to t

In [5]:
print("We have", len(total_results), "total results")

We have 67510 total results


In [6]:
#output metadata records to one giant file
#explore: also output pickle file?
with open(f'{path}zenodo_API_metadata_results_{filetime}.json', 'w') as f:
    json.dump(total_results, f)

## Draft code cells

In [ ]:
#make a text file with a record id on each line, so zenodo_get can be used later to download files
#https://pytutorial.com/python-dump-list-to-file/    
#print(record_id_list)    
with open(f'{path}record_ids_{filetime}.txt', 'w') as file:
    for item in record_id_list:
        file.write(f"{item}\n")

In [ ]:
#loop through the metadata records to extract the record id values
#also write individual metadata files for each record
record_id_list = []

for record in total_results:
    record_id = record['id']
    #put the id in a text file or some object that will then become the text file
    record_id_list.append(record_id)
    #write out a json metadata file for a single record, put it in a directory named for the id? May do this later.
    # with open(f'outputs/{record_id}_metadata_{filetime}.json', 'w') as f:
    #    json.dump(record, f)

In [ ]:
# for date in date_ranges:
#     url = 'https://zenodo.org/api/records'
#     params = {'q': f'filetype:(csv OR xlsx OR tsv OR xls OR ods OR xlsb) AND doi:zenodo AND publicationdate:{date}',
#         'size': '100'
#         #"access_token": ACCESS_TOKEN
#           #test q 'q': 'filetype: xlsb AND doi:zenodo'
#        }
#     response = requests.get(url, params = params)
#     print(response.url)

In [ ]:
#print(record_id_list)

In [ ]:
# for date in date_ranges:
#     print(date)

In [ ]:
#https://www.askpython.com/python/dictionary/create-nested-dictionary-via-for-loop

file_sizes = []

with open('zenodo_snippet.json', 'r') as f:
    data = f.read()

filesize_total = 0
zenodo_dict = json.loads(data)
for rec in zenodo_dict:
    #print(rec['files'])
    files = rec['files']
    record_id = rec['id']
    for file in files:
        match = re.match(r'.*(\.csv|\.xlsx|\.tsv|\.xls|\.ods|\.xlsb)',file['key'])
        print(match)
        file_info = {}
        if match:
            file_info['recordid'] = record_id
            file_info['filename'] = file['key']
            file_info['filetype'] = match.group(1)
            filesize = int(file['size'])
            file_info['filesize'] = filesize
            filesize_total = filesize_total + filesize
            file_sizes.append(file_info)
        else:
            pass
print(filesize_total)
print(file_sizes)
# for record in total_results:
    # print(record['files'])

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(file_sizes)
print(df)

In [ ]:
for rec in total_results:
    #print(rec['files'])
    files = rec['files']
    for file in files:
        match = re.match(r'.*(\.csv|xlsx|tsv|xls|ods|xlsb)',file['key'])
        #print(match)
        if match:
            filesize = int(file['size'])
            filesize_total = filesize_total + filesize
        else:
            pass

In [ ]:
print(filesize_total)
terabytes_size = filesize_total / (1024 * 1024 * 1024 * 1024)
formatted_size = "{:.2f} TB".format(terabytes_size)
print(formatted_size)

The rest of this is various code that failed or I don't need anymore

In [ ]:
#set up API details
# url = 'https://zenodo.org/api/records'
# params = {'q': 'filetype:(csv OR xlsx OR tsv OR xls OR ods OR xlsb) AND doi:zenodo AND publicationdate:[2013 TO 2014}',
#         'size': '100',
#         'access_token': ACCESS_TOKEN
#           #test q 'q': 'filetype: xlsb AND doi:zenodo'
#        }

#https://zenodo.org/api/records?q=filetype%3Acsv+AND+doi%3Azenodo+AND+publicationdate%3A2013

In [ ]:
# response = requests.get(url, params = params)
# print(response.url)

In [ ]:
# tried https://jonathansoma.com/lede/foundations-2018/classes/apis/multiple-pages-of-data-from-apis/
# ran into problems, looked at https://stackoverflow.com/questions/50708172/keyerror-in-while-loop
# still having problems, trying https://farnamdata.com/api-pagination/
# what finally worked was this: https://viniciusdesouza-becker.medium.com/making-a-request-in-a-paginated-api-using-python-7be6279c2aca (see https://gist.github.com/dados-exact/8c8deadb7c9c5d181de4e9c52291f57d#file-main-py)
# but had to take initial response out of the while loop


# results = response.json()
# print(f'current page: {response.url}')
# print(f'the links on this current page are: {results["links"]}')
# total_results.extend(results['hits']['hits'])
# print(f'{len(results["hits"]["hits"])} records were added to the big list')
# nexturl = results['links']['next']
# #nextparams = {'access_token': ACCESS_TOKEN}
# print(f'setting next page: {nexturl}')


In [ ]:

#a while loop that gets all the pages of API results using the 'next' value in the result
# while True:
#     response = requests.get(nexturl)
#     print(f'current page {response.url}')
#     if response.status_code != 200:
#         raise ValueError('Request failed with status code ' + str(response.status_code))
#         print(response.url)
#     results = response.json()
#     print(f'the links on this current page are: {results["links"]}')
#     total_results.extend(results['hits']['hits'])
#     print(f'{len(results["hits"]["hits"])} records were added to the big list')
#     if 'next' in results['links'].keys():
#         nexturl = results['links']['next']
#         print(f'setting next page: {nexturl}')
#     else:
#         break

In [ ]:
# print("We have", len(total_results), "total results")

In [ ]:
#output metadata records to one giant file
#explore: also output pickle file?
# with open(f'{path}zenodo_API_metadata_results_{filetime}.json', 'w') as f:
#     json.dump(total_results, f)

In [ ]:
# response = requests.get(url, params = params)
# print(response.url)

In [ ]:
#print(f'{path}record_ids_{filetime}.txt')

In [ ]:
# print('downloading first page of search results')
# response = requests.get(url, params = params)
# print(response.url)
# print(response.links)
# results = response.json()

In [ ]:
# print(results['links'].keys())
# if 'next' in results['links'].keys():
#     print("yay")
# else:
#     print("boo")

In [ ]:
# # Store the first page of results
# total_results = total_results + results['hits']['hits']
# #print(total_results)

In [ ]:
# response = requests.get(results['links']['next'])
# results = response.json()
# print(response.url)
# # Store the current page of results
# total_results = total_results + results['hits']['hits']
# #print(total_results)

In [ ]:
#from https://jonathansoma.com/lede/foundations-2018/classes/apis/multiple-pages-of-data-from-apis/
# # Start with an empty list
# total_results = []

# # Grab the search results
# print("Downloading the original search results")
# response = requests.get("https://swapi.co/api/people/?search=a")
# data = response.json()

# # Store the first page of results
# total_results = total_results + data['results']

# # While data['next'] isn't empty, let's download the next page, too
# while data['next'] is not None:
#     print("Next page found, downloading", data['next'])
#     response = requests.get(data['next'])
#     data = response.json()
#     # Store the current page of results
#     total_results = total_results + data['results']

# print("We have", len(total_results), "total results")

In [ ]:
# response = requests.get('https://zenodo.org/api/records',
#                         params={'q': 'filetype:(csv OR xlsx OR tsv OR xls OR ods OR xslb) AND doi:zenodo',
#                                 'size': '10',
#                                 #'access_token': ACCESS_TOKEN
#                                 # regular search: https://zenodo.org/search?q=&f=file_type%3Axlsx&f=file_type%3Acsv&f=file_type%3Atsv&f=file_type%3Axls&f=file_type%3Axlsb&f=file_type%3Aods&l=list&p=1&s=10&sort=newest
#                                })
# results = response.json()

In [ ]:
# with open('spreadsheet-metadata_%s.json' %filetime, 'w') as f:
#     f.write(response.text)

In [ ]:
# print(response.url)

In [ ]:
# for record in results['hits']['hits']:
#     record_id = record['id']
#     with open('metadata_%s.json' %record_id, 'w') as f:
#         json.dump(record, f, ensure_ascii=False)
#     # # print(record['metadata']['title'])
#     # # print(record['files'])
#     # for file in record['files']:
#     #     filename = file['key']
#     #     #print(filename)
#     #     if re.search(r'\.(csv|tsv|xlsx|xls|xlsb|ods)', filename):
#     #         print("spreadsheet file found:", filename)

In [ ]:
# len("Output_Area_to_LSOA_to_MSOA_to_Local_Authority_District__December_2017__Lookup_with_Area_Classifications_in_Great_Britain.csv")

In [ ]:
# for filetype in results['aggregations']['file_type']['buckets']:
#     print(filetype['key'],filetype['doc_count'])